In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import chart_studio.plotly as py

In [ ]:
from dataclasses import dataclass
import sys
from itertools import groupby
from random import random, choice


@dataclass
class Person:
    infected_on_day: int = -999

    def __post_init__(self):
        if random() < 0.1:
            self.outcome = "dead"
        else:
            self.outcome = "recovered"

    def expose(self, today):
        if self.infected_on_day == -999:
            self.infected_on_day = today

    def is_infected(self, today):
        if self.infected_on_day == -999:
            return False
        days_since_infection = today - self.infected_on_day
        return days_since_infection < 14

    def is_contagious(self, today):
        if self.infected_on_day == -999:
            return False
        days_since_infection = today - self.infected_on_day
        return 1 < days_since_infection < 10

    def state(self, today):
        if self.is_infected(today=today):
            if self.is_contagious(today=today):
                return "contagious"
            return "not contagious"
        if self.infected_on_day == -999:
            return "uninfected"
        return self.outcome


@dataclass
class Sim:
    today: int = 0
    population: int = 10000
    initial_exposed: int = 100
    infection_chance: float = 0.10
    previous_infection_chance: float = 0.30
    max_days: int = 100

    def tick(self, chance):
        self.today = self.today + 1
        for person in self.people:
            if person.is_contagious(today=self.today) and random() < chance:
                choice(self.people).expose(today=self.today)

    def run(self):
        infected = 0
        while infected == 0:
            self.people = []
            for _ in range(self.population):
                self.people.append(Person())

            result = []
            self.people[0].expose(today=self.today)  # patient zero
            for _ in range(self.max_days):
                self.tick(chance=self.previous_infection_chance)
                keyfunc = lambda x: x.state(today=self.today)
                summ = groupby(sorted(self.people, key=keyfunc), keyfunc)
                infected = sum([len(list(v)) for k, v in summ if "contagious" in k])
                if infected >= self.initial_exposed:
                    break

        # reset day count
        for person in self.people:
            if person.infected_on_day > -999:
                person.infected_on_day -= self.today
        self.today = 0

        for _ in range(self.max_days):
            self.tick(chance=self.infection_chance)
            keyfunc = lambda x: x.state(today=self.today)
            summ = groupby(sorted(self.people, key=keyfunc), keyfunc)
            infected = sum([len(list(v)) for k, v in summ if "contagious" in k])
            result.append(infected)
        return result


In [ ]:
max_days=200
n_sims = 1

In [ ]:
dates = pd.date_range('20200301', periods=max_days, freq='d')

In [ ]:
# runs = pd.DataFrame(index=dates, data={f"sim {_}": Sim(max_days=max_days, infection_chance=0.10).run() for _ in range(n_sims)})

In [ ]:
series = pd.Series(name="sim", index=dates, data=Sim(max_days=max_days, infection_chance=0.05).run())

In [ ]:
import plotly.express as px
fig = px.line(series, y="sim", x=series.index)
fig.show()